* Phase 1: Dataset Understanding & Selection
* Phase 2: Data Cleaning & Aggregation
* Phase 3: Feature Engineering
* Phase 4: Model Training (Multiple Models)
* Phase 5: Streamlit Integration
* Phase 6: Evaluation & Deployment

# Phase 1: Dataset Understanding & Selection

In [ ]:
!pip install -q kaggle timm albumentations seaborn

In [ ]:
from google.colab import files
files.upload()  # upload kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"amritguptaaw","key":"04f64164448187e366a922cdaf4cd156"}'}

In [ ]:
import os, zipfile

os.makedirs("/root/.kaggle", exist_ok=True)
!cp /content/kaggle.json /root/.kaggle/
os.chmod("/root/.kaggle/kaggle.json", 600)

In [ ]:
import os
import zipfile
import urllib.request

# Create data directory
os.makedirs("/content/UCI", exist_ok=True)

# UCI dataset URL
uci_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00321/LD2011_2014.txt.zip"
zip_path = "/content/UCI/LD2011_2014.txt.zip"

# Download
urllib.request.urlretrieve(uci_url, zip_path)

print("Download completed.")

Download completed.


In [ ]:
# 1.2 Download the dataset (example: Michael Looney’s dataset)
!kaggle datasets download -d michaelrlooney/electricity-load-diagrams-2011-2014 -p /content

Dataset URL: https://www.kaggle.com/datasets/michaelrlooney/electricity-load-diagrams-2011-2014
License(s): Attribution 4.0 International (CC BY 4.0)
 57% 144M/251M [00:00<00:00, 1.49GB/s]
100% 251M/251M [00:00<00:00, 882MB/s] 


In [ ]:
# 1.3 Unzip
zip_path = "/content/electricity-load-diagrams-2011-2014.zip"
with zipfile.ZipFile(zip_path, "r") as z:
    z.extractall("/content")

In [ ]:
import pandas as pd

file_path = "/content/LD2011_2014.txt"

In [ ]:
# 1.4 Load full dataset
df_raw = pd.read_csv(
    file_path,
    sep=";",          # semicolon
    decimal=",",      # comma as decimal separator
    low_memory=False
)

print("Shape:", df_raw.shape)
print(df_raw.columns[:10])
print(df_raw.head())

Shape: (140256, 371)
Index(['Unnamed: 0', 'MT_001', 'MT_002', 'MT_003', 'MT_004', 'MT_005',
       'MT_006', 'MT_007', 'MT_008', 'MT_009'],
      dtype='object')
            Unnamed: 0  MT_001  MT_002  MT_003  MT_004  MT_005  MT_006  \
0  2011-01-01 00:15:00     0.0     0.0     0.0     0.0     0.0     0.0   
1  2011-01-01 00:30:00     0.0     0.0     0.0     0.0     0.0     0.0   
2  2011-01-01 00:45:00     0.0     0.0     0.0     0.0     0.0     0.0   
3  2011-01-01 01:00:00     0.0     0.0     0.0     0.0     0.0     0.0   
4  2011-01-01 01:15:00     0.0     0.0     0.0     0.0     0.0     0.0   

   MT_007  MT_008  MT_009  ...  MT_361  MT_362  MT_363  MT_364  MT_365  \
0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0     0.0   
1     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0     0.0   
2     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0     0.0   
3     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0     0.0   
4     0.0     0.0     0

In [ ]:
# 1.5 Convert first column to datetime index
datetime_col = df_raw.columns[0]   # usually "Datetime" or similar
df_raw[datetime_col] = pd.to_datetime(df_raw[datetime_col])
df_raw = df_raw.set_index(datetime_col)

print("Final shape:", df_raw.shape)
print("Date range:", df_raw.index.min(), "→", df_raw.index.max())

Final shape: (140256, 370)
Date range: 2011-01-01 00:15:00 → 2015-01-01 00:00:00


In [ ]:
# Quick peek at the data
print("head")
print(df_raw.head())      # first 5 rows
print(" tail")
print(df_raw.tail())      # last 5 rows
print(df_raw.info())      # column details
print(df_raw.describe())  # summary statistics

head
                     MT_001  MT_002  MT_003  MT_004  MT_005  MT_006  MT_007  \
Unnamed: 0                                                                    
2011-01-01 00:15:00     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2011-01-01 00:30:00     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2011-01-01 00:45:00     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2011-01-01 01:00:00     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2011-01-01 01:15:00     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

                     MT_008  MT_009  MT_010  ...  MT_361  MT_362  MT_363  \
Unnamed: 0                                   ...                           
2011-01-01 00:15:00     0.0     0.0     0.0  ...     0.0     0.0     0.0   
2011-01-01 00:30:00     0.0     0.0     0.0  ...     0.0     0.0     0.0   
2011-01-01 00:45:00     0.0     0.0     0.0  ...     0.0     0.0     0.0   
2011-01-01 01:00:00     0.0     0.0     0.0  ...     0.0     

In [ ]:
print("Shape:", df_raw.shape)
print("Start:", df_raw.index.min())
print("End:", df_raw.index.max())
print("Columns:", list(df_raw.columns[:10]), "...")  # first 10 column names

Shape: (140256, 370)
Start: 2011-01-01 00:15:00
End: 2015-01-01 00:00:00
Columns: ['MT_001', 'MT_002', 'MT_003', 'MT_004', 'MT_005', 'MT_006', 'MT_007', 'MT_008', 'MT_009', 'MT_010'] ...



# Phase 2 – Cleaning & aggregation (household + city, hourly + daily)

In [ ]:
#2.1 Convert kW → kWh (15-min → energy)
df_energy = df_raw.copy().astype(float)
df_energy = df_energy / 4.0     # kWh per 15-min interval
df_energy.columns = [c + "_kwh" for c in df_energy.columns]

In [ ]:
# 2.2 Choose main household
household_col = "MT_001_kwh"    # you can change to MT_002_kwh etc.
df_household_15min = df_energy[[household_col]].rename(columns={household_col: "energy_kwh"})

In [ ]:
# 2.3 Household hourly
df_hourly_household = (
    df_household_15min
    .resample("H")
    .sum()
    .reset_index()
)
df_hourly_household.rename(columns={"index": "datetime"}, inplace=True)

/tmp/ipython-input-152584804.py:4: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  .resample("H")


In [ ]:
# 2.4 Household daily
df_daily_household = (
    df_household_15min
    .resample("D")
    .sum()
    .reset_index()
)
df_daily_household.rename(columns={"index": "datetime"}, inplace=True)

In [ ]:
# 2.5 City 15-min (sum across all clients)
df_city_15min = df_energy.sum(axis=1).to_frame(name="energy_kwh")

In [ ]:
# 2.6 City hourly
df_hourly_city = (
    df_city_15min
    .resample("H")
    .sum()
    .reset_index()
)
df_hourly_city.rename(columns={"index": "datetime"}, inplace=True)

/tmp/ipython-input-3429100563.py:4: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  .resample("H")


In [ ]:
# 2.7 City daily
df_daily_city = (
    df_city_15min
    .resample("D")
    .sum()
    .reset_index()
)
df_daily_city.rename(columns={"index": "datetime"}, inplace=True)

In [ ]:
# 2.8 Add metadata columns
def add_metadata(df, level, granularity):
    df = df.copy()
    df["level"] = level
    df["granularity"] = granularity
    return df

hourly_household = add_metadata(df_hourly_household, "household", "hourly")
daily_household  = add_metadata(df_daily_household,  "household", "daily")
hourly_city      = add_metadata(df_hourly_city,      "city",      "hourly")
daily_city       = add_metadata(df_daily_city,       "city",      "daily")

In [ ]:
# 2.9 Master dataset
energy_master = pd.concat(
    [hourly_household, daily_household, hourly_city, daily_city],
    ignore_index=True
)

import os
os.makedirs("data", exist_ok=True)

In [ ]:
hourly_household.to_csv("data/hourly_household.csv", index=False)
daily_household.to_csv("data/daily_household.csv", index=False)
hourly_city.to_csv("data/hourly_city.csv", index=False)
daily_city.to_csv("data/daily_city.csv", index=False)
energy_master.to_csv("data/energy_master_dataset.csv", index=False)

In [ ]:
energy_master.head()
energy_master.tail()

,Unnamed: 0,energy_kwh,level,granularity
73049,2014-12-28,4.734025e+06,city,daily
73050,2014-12-29,4.830825e+06,city,daily
73051,2014-12-30,4.831542e+06,city,daily
73052,2014-12-31,4.340759e+06,city,daily
73053,2015-01-01,3.105355e+04,city,daily


In [ ]:
print("Shape:", energy_master.shape)
print("Start:", energy_master.index.min())
print("End:", energy_master.index.max())

Shape: (73054, 4)
Start: 0
End: 73053


# Phase 3 – Feature engineering (hourly household)

In [ ]:
import pandas as pd

# 1) See what columns actually exist
df_check = pd.read_csv("data/hourly_household.csv")
print(df_check.columns)

# 2) Now load with correct parse_dates argument based on real column name
df = pd.read_csv("data/hourly_household.csv", parse_dates=[0])  # first column is datetime
df = df.sort_values(df.columns[0]).reset_index(drop=True)

# If needed, rename for consistency
df.rename(columns={df.columns[0]: "datetime"}, inplace=True)

df.head()
print(df.dtypes)


Index(['Unnamed: 0', 'energy_kwh', 'level', 'granularity'], dtype='object')
datetime       datetime64[ns]
energy_kwh            float64
level                  object
granularity            object
dtype: object


In [ ]:
df = pd.read_csv("data/hourly_household.csv", parse_dates=[0])
df.rename(columns={df.columns[0]: "datetime"}, inplace=True)
df = df.sort_values("datetime").reset_index(drop=True)

In [ ]:
# 3.1 Time features
df["hour"] = df["datetime"].dt.hour
df["day"] = df["datetime"].dt.day
df["month"] = df["datetime"].dt.month
df["day_of_week"] = df["datetime"].dt.dayofweek
df["is_weekend"] = df["day_of_week"].isin([5, 6]).astype(int)

In [ ]:
# 3.2 Cyclical encodings
import numpy as np

df["hour_sin"] = np.sin(2 * np.pi * df["hour"] / 24)
df["hour_cos"] = np.cos(2 * np.pi * df["hour"] / 24)
df["dow_sin"]  = np.sin(2 * np.pi * df["day_of_week"] / 7)
df["dow_cos"]  = np.cos(2 * np.pi * df["day_of_week"] / 7)

In [ ]:
# 3.3 Lag features
df["lag_1"]   = df["energy_kwh"].shift(1)
df["lag_24"]  = df["energy_kwh"].shift(24)
df["lag_168"] = df["energy_kwh"].shift(168)

In [ ]:
# 3.4 Rolling means
df["rolling_24_mean"]  = df["energy_kwh"].rolling(window=24).mean()
df["rolling_168_mean"] = df["energy_kwh"].rolling(window=168).mean()

In [ ]:
# 3.5 Drop NA
df_fe = df.dropna().reset_index(drop=True)

In [ ]:
# 3.6 Optional log-transform target
df_fe["energy_log"] = np.log1p(df_fe["energy_kwh"])

In [ ]:
df_fe.to_csv("data/hourly_household_FEATURES.csv", index=False)
df_fe.head()

,datetime,energy_kwh,level,granularity,hour,day,month,day_of_week,is_weekend,hour_sin,hour_cos,dow_sin,dow_cos,lag_1,lag_24,lag_168,rolling_24_mean,rolling_168_mean,energy_log
0,2011-01-08 00:00:00,0.0,household,hourly,0,8,1,5,1,0.000000,1.000000,-0.974928,-0.222521,0.0,0.0,0.0,0.0,0.0,0.0
1,2011-01-08 01:00:00,0.0,household,hourly,1,8,1,5,1,0.258819,0.965926,-0.974928,-0.222521,0.0,0.0,0.0,0.0,0.0,0.0
2,2011-01-08 02:00:00,0.0,household,hourly,2,8,1,5,1,0.500000,0.866025,-0.974928,-0.222521,0.0,0.0,0.0,0.0,0.0,0.0
3,2011-01-08 03:00:00,0.0,household,hourly,3,8,1,5,1,0.707107,0.707107,-0.974928,-0.222521,0.0,0.0,0.0,0.0,0.0,0.0
4,2011-01-08 04:00:00,0.0,household,hourly,4,8,1,5,1,0.866025,0.500000,-0.974928,-0.222521,0.0,0.0,0.0,0.0,0.0,0.0


# Phase 4 – Model training (multiple models on hourly household)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

In [ ]:
# 4.1 Feature matrix and target
FEATURES = [
    "hour", "day", "month", "day_of_week", "is_weekend",
    "hour_sin", "hour_cos", "dow_sin", "dow_cos",
    "lag_1", "lag_24", "lag_168",
    "rolling_24_mean", "rolling_168_mean"
]

X = df_fe[FEATURES]
y = df_fe["energy_log"]   # or "energy_kwh" if you skip log

In [ ]:
# 4.2 Time-based split (80/20)
split_index = int(len(df_fe) * 0.8)
X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]

def evaluate_model(y_true, y_pred_log, label):
    y_true_actual = np.expm1(y_true)
    y_pred_actual = np.expm1(y_pred_log)

    mae = mean_absolute_error(y_true_actual, y_pred_actual)
    rmse = np.sqrt(mean_squared_error(y_true_actual, y_pred_actual))
    r2 = r2_score(y_true_actual, y_pred_actual)

    print(f"{label} Performance")
    print(f"MAE:  {mae:.3f}")
    print(f"RMSE: {rmse:.3f}")
    print(f"R²:   {r2:.3f}")
    return mae, rmse, r2

### BaseModel: Linear Regression

In [ ]:
# 4.3 Linear Regression
lr = LinearRegression()
lr.fit(X_train, y_train)
lr_preds = lr.predict(X_test)
lr_mae, lr_rmse, lr_r2 = evaluate_model(y_test, lr_preds, "Linear Regression")

# 4.4 Random Forest
rf = RandomForestRegressor(
    n_estimators=200,
    max_depth=15,
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train)
rf_preds = rf.predict(X_test)
rf_mae, rf_rmse, rf_r2 = evaluate_model(y_test, rf_preds, "Random Forest")

Linear Regression Performance
MAE:  1.976
RMSE: 3.616
R²:   0.617
Random Forest Performance
MAE:  1.026
RMSE: 2.173
R²:   0.862


### Model: Random Forest

In [ ]:
# 4.4 Random Forest
rf = RandomForestRegressor(
    n_estimators=200,
    max_depth=15,
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train)
rf_preds = rf.predict(X_test)
rf_mae, rf_rmse, rf_r2 = evaluate_model(y_test, rf_preds, "Random Forest")

Random Forest Performance
MAE:  1.026
RMSE: 2.173
R²:   0.862


### Model: XGBoost

In [ ]:
# 4.5 XGBoost
xgb = XGBRegressor(
    n_estimators=800,
    learning_rate=0.03,
    max_depth=8,
    subsample=0.9,
    colsample_bytree=0.9,
    objective="reg:squarederror",
    random_state=42,
    n_jobs=-1
)
xgb.fit(X_train, y_train)
xgb_preds = xgb.predict(X_test)
xgb_mae, xgb_rmse, xgb_r2 = evaluate_model(y_test, xgb_preds, "XGBoost")

XGBoost Performance
MAE:  1.044
RMSE: 2.187
R²:   0.860


In [ ]:
# 4.6 Results table
import pandas as pd

results = pd.DataFrame({
    "Model": ["Linear Regression", "Random Forest", "XGBoost"],
    "MAE":   [lr_mae, rf_mae, xgb_mae],
    "RMSE":  [lr_rmse, rf_rmse, xgb_rmse],
    "R2":    [lr_r2, rf_r2, xgb_r2],
})
results

,Model,MAE,RMSE,R2
0,Linear Regression,1.976470,3.615682,0.617277
1,Random Forest,1.025695,2.173137,0.861746
2,XGBoost,1.043572,2.186539,0.860035


In [ ]:
import joblib
import os

os.makedirs("models", exist_ok=True)

# 1. Save individual models
joblib.dump(lr,  "models/household_hourly_linear_regression.joblib")
joblib.dump(rf,  "models/household_hourly_random_forest.joblib")
joblib.dump(xgb, "models/household_hourly_xgboost.joblib")

# 2. Save metrics/results table
results.to_csv("models/household_hourly_model_results.csv", index=False)

print("Saved household hourly models and results.")

Saved household hourly models and results.


In [ ]:
# Choose best model by highest R2
best_row = results.loc[results["R2"].idxmax()]
best_model_name = best_row["Model"]
print("Best model:", best_model_name)

model_map = {
    "Linear Regression": lr,
    "Random Forest": rf,
    "XGBoost": xgb,
}

best_model = model_map[best_model_name]
joblib.dump(best_model, "models/household_hourly_BEST.joblib")
print("Saved best model to models/household_hourly_BEST.joblib")

Best model: Random Forest
Saved best model to models/household_hourly_BEST.joblib


In [ ]:
import shutil
import os

# Make sure folders exist
print("Data exists:", os.path.exists("data"))
print("Models exists:", os.path.exists("models"))

# 1) Zip data folder
if os.path.exists("data"):
    shutil.make_archive("data_folder_backup", "zip", "data")
    print("Created data_folder_backup.zip")

# 2) Zip models folder
if os.path.exists("models"):
    shutil.make_archive("models_folder_backup", "zip", "models")
    print("Created models_folder_backup.zip")

Data exists: True
Models exists: True
Created data_folder_backup.zip
Created models_folder_backup.zip


In [ ]:
from google.colab import files

# Download the ZIP files to your local machine
files.download("data_folder_backup.zip")
files.download("models_folder_backup.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Phase: City-level hourly model

## Feature Enginnering

In [46]:
import pandas as pd
import numpy as np

df_city = pd.read_csv("/content/data/hourly_city.csv")

# 🔍 Fix datetime column safely
if "datetime" not in df_city.columns:
    df_city = df_city.rename(columns={df_city.columns[0]: "datetime"})

df_city["datetime"] = pd.to_datetime(df_city["datetime"])
df_city = df_city.sort_values("datetime").reset_index(drop=True)

# Rename energy column if needed
if "energykwh" not in df_city.columns:
    df_city = df_city.rename(columns={"energy_kwh": "energykwh"})

# Time features
df_city["hour"] = df_city["datetime"].dt.hour
df_city["day"] = df_city["datetime"].dt.day
df_city["month"] = df_city["datetime"].dt.month
df_city["day_of_week"] = df_city["datetime"].dt.dayofweek
df_city["is_weekend"] = df_city["day_of_week"].isin([5, 6]).astype(int)

# Cyclical encodings
df_city["hour_sin"] = np.sin(2 * np.pi * df_city["hour"] / 24)
df_city["hour_cos"] = np.cos(2 * np.pi * df_city["hour"] / 24)
df_city["dow_sin"]  = np.sin(2 * np.pi * df_city["day_of_week"] / 7)
df_city["dow_cos"]  = np.cos(2 * np.pi * df_city["day_of_week"] / 7)

# ✅ LOG AFTER AGGREGATION
df_city["energy_log"] = np.log1p(df_city["energykwh"])

# Lags & rolling (LOG SPACE)
df_city["lag_1"]   = df_city["energy_log"].shift(1)
df_city["lag_24"]  = df_city["energy_log"].shift(24)
df_city["lag_168"] = df_city["energy_log"].shift(168)

df_city["rolling_24_mean"]  = df_city["energy_log"].rolling(24).mean()
df_city["rolling_168_mean"] = df_city["energy_log"].rolling(168).mean()

df_city_fe = df_city.dropna().reset_index(drop=True)


In [49]:
df_city_fe.head()

,datetime,energykwh,level,granularity,hour,day,month,day_of_week,is_weekend,hour_sin,hour_cos,dow_sin,dow_cos,energy_log,lag_1,lag_24,lag_168,rolling_24_mean,rolling_168_mean
0,2011-01-08 00:00:00,86403.301624,city,hourly,0,8,1,5,1,0.000000,1.000000,-0.974928,-0.222521,11.366793,11.622771,11.358816,10.854480,11.660132,11.582337
1,2011-01-08 01:00:00,72534.984171,city,hourly,1,8,1,5,1,0.258819,0.965926,-0.974928,-0.222521,11.191838,11.366793,11.202301,11.102633,11.659696,11.582867
2,2011-01-08 02:00:00,69325.030607,city,hourly,2,8,1,5,1,0.500000,0.866025,-0.974928,-0.222521,11.146576,11.191838,11.163999,11.097138,11.658970,11.583162
3,2011-01-08 03:00:00,68554.184257,city,hourly,3,8,1,5,1,0.707107,0.707107,-0.974928,-0.222521,11.135394,11.146576,11.159716,11.106122,11.657956,11.583336
4,2011-01-08 04:00:00,69033.522311,city,hourly,4,8,1,5,1,0.866025,0.500000,-0.974928,-0.222521,11.142362,11.135394,11.140194,11.081597,11.658047,11.583698


## MODEL TRAINING

In [48]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

FEATURES_CITY = [
    "hour", "day", "month", "day_of_week", "is_weekend",
    "hour_sin", "hour_cos", "dow_sin", "dow_cos",
    "lag_1", "lag_24", "lag_168",
    "rolling_24_mean", "rolling_168_mean"
]

# ===============================
# FEATURES & TARGET
# ===============================
Xc = df_city_fe[FEATURES_CITY]
yc = df_city_fe["energy_log"]

# ===============================
# TIME-BASED SPLIT (NO LEAKAGE)
# ===============================
split = int(len(df_city_fe) * 0.8)

Xc_train, Xc_test = Xc.iloc[:split], Xc.iloc[split:]
yc_train, yc_test = yc.iloc[:split], yc.iloc[split:]

# ===============================
# TRAIN MODEL
# ===============================
city_model = RandomForestRegressor(
    n_estimators=300,
    max_depth=20,
    random_state=42,
    n_jobs=-1
)

city_model.fit(Xc_train, yc_train)

# ===============================
# EVALUATION (LOG → REAL SCALE)
# ===============================
log_preds = city_model.predict(Xc_test)

preds = np.expm1(log_preds)
actuals = np.expm1(yc_test)

mae = mean_absolute_error(actuals, preds)
rmse = np.sqrt(mean_squared_error(actuals, preds))
r2 = r2_score(actuals, preds)

print(f"MAE : {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R2  : {r2:.4f}")


MAE : 4509.38
RMSE: 6997.68
R2  : 0.9930


In [51]:
import os
os.makedirs("models", exist_ok=True)

In [52]:
import joblib
joblib.dump(city_model, "models/city_hourly_TOTAL.joblib")

['models/city_hourly_TOTAL.joblib']

In [54]:
import shutil
shutil.make_archive("city_model", "zip", "models")

'/content/city_model.zip'

In [55]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [56]:
import shutil
shutil.copy(
    "models/city_hourly_TOTAL.joblib",
    "/content/drive/MyDrive/city_hourly_TOTAL.joblib"
)

'/content/drive/MyDrive/city_hourly_TOTAL.joblib'